In [1]:
import pandas as pd
import requests as rq

In [2]:
# APIURL = "https://schoolsdb2.herokuapp.com/cdata/"
APIURL = "http://localhost:8000/cdata/"

In [3]:
schools = rq.get(APIURL + "dataset/schools/").json()
schools[1000]

{u'city': u'Niantic',
 u'district': u'East Lyme School District',
 u'district_code': u'0450011',
 u'education_program': u'',
 u'fax': u'',
 u'grade_1': u'0',
 u'grade_10': u'0',
 u'grade_11': u'0',
 u'grade_12': u'0',
 u'grade_2': u'0',
 u'grade_3': u'0',
 u'grade_4': u'0',
 u'grade_5': u'1',
 u'grade_6': u'1',
 u'grade_7': u'1',
 u'grade_8': u'1',
 u'grade_9': u'0',
 u'inter_district_magnet': u'',
 u'kindergarten': u'0',
 u'phone': u'860-739-4491',
 u'pre_kindergarten': u'0',
 u'program_type': u'',
 u'school': u'East Lyme Middle School',
 u'school_code': u'0455111',
 u'state': u'CT',
 u'street': u'31 Society Rd.',
 u'type': u'Public Schools',
 u'website': u'',
 u'zip': u'06357'}

In [4]:
drgs = pd.read_csv("../raw/ERGbyDistrict.csv")

drgs.head()

,ERGCode,DistrictName,Unnamed: 2
0,A,DARIEN,NaN
1,A,EASTON,NaN
2,A,NEW CANAAN,NaN
3,A,REDDING,NaN
4,A,REGIONAL SCHOOL DISTRICT 09 ( Easton Redding),NaN


In [5]:
def find_contains(needle, haystack_df, key):
    df = haystack_df.copy()
    df[key] = df[key].str.upper()
    needle = needle.upper()
    
    print needle, key

    return haystack_df[haystack_df[key].str.startswith(needle)]

def find_district(name):
    return find_contains(name.upper().replace("SCHOOL DISTRICT","").strip(), drgs, "DistrictName")
            

In [6]:
find_district

<function __main__.find_district>

In [7]:
def is_district( x ):
    if x is None: 
        return False
    if "type" not in x or x["type"] is None: 
        return False
    
    return "DISTRICT" in x["type"].upper()
    
just_districts = filter(lambda x: is_district(x), schools)
just_districts[0]

{u'city': u'Woodstock',
 u'district': u'Woodstock School District',
 u'district_code': u'1690011',
 u'education_program': u'',
 u'fax': u'',
 u'grade_1': u'',
 u'grade_10': u'',
 u'grade_11': u'',
 u'grade_12': u'',
 u'grade_2': u'',
 u'grade_3': u'',
 u'grade_4': u'',
 u'grade_5': u'',
 u'grade_6': u'',
 u'grade_7': u'',
 u'grade_8': u'',
 u'grade_9': u'',
 u'inter_district_magnet': u'',
 u'kindergarten': u'',
 u'phone': u'860-928-7453',
 u'pre_kindergarten': u'',
 u'program_type': u'',
 u'school': u'Woodstock School District',
 u'school_code': u'1690011',
 u'state': u'CT',
 u'street': u'147 A Route 169',
 u'type': u'Public School Districts',
 u'website': u'www.woodstockschools.net',
 u'zip': u'06281'}

In [8]:
def find_district_code(dist_name):
    return filter(lambda x: x["school"].lower().startswith(dist_name.lower()), just_districts )
    
#find_district_code("REGIONAL SCHOOL DISTRICT")
find_district_code

<function __main__.find_district_code>

In [9]:
def add_district_id(df):
    
    ret = df.copy()
    
    district_codes = []
    district_name_check = []
    for i, r in ret.iterrows():
        distname = (" ").join(r["DistrictName"].replace("("," (").split(" ")[:4])
        matches = find_district_code(distname)
        if (len(matches) == 1):
            #print distname, matches[0]["school"]
            district_codes.append(matches[0]["district_code"])
            district_name_check.append(matches[0]["school"])
            
        elif (len(matches) > 1):
            print distname, "TOO MANY", len(matches)
            district_codes.append(None)
            district_name_check.append(None)

        elif (len(matches) < 1):
            print distname, "NO MATCHES"
            district_codes.append(None)
            district_name_check.append(None)

            
    ret["school_code"] = pd.Series(district_codes).values
    ret["district_name_check"] = pd.Series(district_name_check).values

    return ret

with_codes = add_district_id(drgs)    

WINDSOR TOO MANY 2
WOODSTOCK TOO MANY 2
NORWICH TOO MANY 2
STAMFORD TOO MANY 3
BRIDGEPORT TOO MANY 2


In [10]:
with_codes.columns = "drg","drg_distname","NA","school_code","district_name_check"

In [11]:
with_codes = with_codes[["drg","drg_distname","school_code","district_name_check"]]

In [12]:
find_district_code("HARTFORD")

[{u'city': u'Hartford',
  u'district': u'Hartford School District',
  u'district_code': u'0640011',
  u'education_program': u'',
  u'fax': u'',
  u'grade_1': u'',
  u'grade_10': u'',
  u'grade_11': u'',
  u'grade_12': u'',
  u'grade_2': u'',
  u'grade_3': u'',
  u'grade_4': u'',
  u'grade_5': u'',
  u'grade_6': u'',
  u'grade_7': u'',
  u'grade_8': u'',
  u'grade_9': u'',
  u'inter_district_magnet': u'',
  u'kindergarten': u'',
  u'phone': u'860-695-8000',
  u'pre_kindergarten': u'',
  u'program_type': u'',
  u'school': u'Hartford School District',
  u'school_code': u'0640011',
  u'state': u'CT',
  u'street': u'249 High Street',
  u'type': u'Public School Districts',
  u'website': u'www.hartfordschools.org/',
  u'zip': u'06103'}]

# Fix exceptions

All but six scholls came over cleanly. Here are the problem ones:

In [13]:
with_codes[with_codes["school_code"].isnull()]

,drg,drg_distname,school_code,district_name_check
83,D,WINDSOR,None,None
117,E,WOODSTOCK,None,None
159,H,NORWICH,None,None
160,H,STAMFORD,None,None
162,I,BRIDGEPORT,None,None


## 1. Windsor, not Windsor Locks

In [14]:
find_district_code("WINDSOR")

[{u'city': u'Windsor',
  u'district': u'Windsor School District',
  u'district_code': u'1640011',
  u'education_program': u'',
  u'fax': u'860-687-2009',
  u'grade_1': u'',
  u'grade_10': u'',
  u'grade_11': u'',
  u'grade_12': u'',
  u'grade_2': u'',
  u'grade_3': u'',
  u'grade_4': u'',
  u'grade_5': u'',
  u'grade_6': u'',
  u'grade_7': u'',
  u'grade_8': u'',
  u'grade_9': u'',
  u'inter_district_magnet': u'',
  u'kindergarten': u'',
  u'phone': u'860-687-2000',
  u'pre_kindergarten': u'',
  u'program_type': u'',
  u'school': u'Windsor School District',
  u'school_code': u'1640011',
  u'state': u'CT',
  u'street': u'601 Matianuck Avenue',
  u'type': u'Public School Districts',
  u'website': u'www.windsorct.org',
  u'zip': u'06095'},
 {u'city': u'Windsor Locks',
  u'district': u'Windsor Locks School District',
  u'district_code': u'1650011',
  u'education_program': u'',
  u'fax': u'',
  u'grade_1': u'',
  u'grade_10': u'',
  u'grade_11': u'',
  u'grade_12': u'',
  u'grade_2': u'',
 

In [15]:
def fix_record(df, drg_distname, record):
    ret = df.copy()
    i = with_codes[with_codes["drg_distname"] == drg_distname].index
    row = with_codes.iloc[i]
            
    ret.iloc[i] = [row["drg"].values[0],
                   row["drg_distname"].values[0],
                   record["district_code"],
                  record["school"]]
    
    return ret
    
fixed = fix_record(with_codes, "WINDSOR", find_district_code("WINDSOR")[0])

In [16]:
fixed[fixed["drg_distname"] == "WINDSOR"]

,drg,drg_distname,school_code,district_name_check
83,D,WINDSOR,1640011,Windsor School District


## WOODSTOCK

In [17]:
# Let's see what's still broken

fixed[fixed["school_code"].isnull()]


,drg,drg_distname,school_code,district_name_check
117,E,WOODSTOCK,None,None
159,H,NORWICH,None,None
160,H,STAMFORD,None,None
162,I,BRIDGEPORT,None,None


In [18]:
find_district_code("WOODSTOCK")

[{u'city': u'Woodstock',
  u'district': u'Woodstock School District',
  u'district_code': u'1690011',
  u'education_program': u'',
  u'fax': u'',
  u'grade_1': u'',
  u'grade_10': u'',
  u'grade_11': u'',
  u'grade_12': u'',
  u'grade_2': u'',
  u'grade_3': u'',
  u'grade_4': u'',
  u'grade_5': u'',
  u'grade_6': u'',
  u'grade_7': u'',
  u'grade_8': u'',
  u'grade_9': u'',
  u'inter_district_magnet': u'',
  u'kindergarten': u'',
  u'phone': u'860-928-7453',
  u'pre_kindergarten': u'',
  u'program_type': u'',
  u'school': u'Woodstock School District',
  u'school_code': u'1690011',
  u'state': u'CT',
  u'street': u'147 A Route 169',
  u'type': u'Public School Districts',
  u'website': u'www.woodstockschools.net',
  u'zip': u'06281'},
 {u'city': u'Woodstock',
  u'district': u'Woodstock Academy District',
  u'district_code': u'9030022',
  u'education_program': u'',
  u'fax': u'',
  u'grade_1': u'',
  u'grade_10': u'',
  u'grade_11': u'',
  u'grade_12': u'',
  u'grade_2': u'',
  u'grade_3'

In [19]:
# Record 0 looks right
fixed = fix_record(fixed, "WOODSTOCK", find_district_code("WOODSTOCK")[0])

In [20]:
fixed[fixed["school_code"].isnull()]

,drg,drg_distname,school_code,district_name_check
159,H,NORWICH,None,None
160,H,STAMFORD,None,None
162,I,BRIDGEPORT,None,None


# 3. NORWICH

In [21]:
find_district_code("NORWICH")

[{u'city': u'Norwich',
  u'district': u'Norwich School District',
  u'district_code': u'1040011',
  u'education_program': u'',
  u'fax': u'',
  u'grade_1': u'',
  u'grade_10': u'',
  u'grade_11': u'',
  u'grade_12': u'',
  u'grade_2': u'',
  u'grade_3': u'',
  u'grade_4': u'',
  u'grade_5': u'',
  u'grade_6': u'',
  u'grade_7': u'',
  u'grade_8': u'',
  u'grade_9': u'',
  u'inter_district_magnet': u'',
  u'kindergarten': u'',
  u'phone': u'860-823-4200',
  u'pre_kindergarten': u'',
  u'program_type': u'',
  u'school': u'Norwich School District',
  u'school_code': u'1040011',
  u'state': u'CT',
  u'street': u'90 Town Street',
  u'type': u'Public School Districts',
  u'website': u'www.norwichpublicschools.org',
  u'zip': u'06360'},
 {u'city': u'Norwich',
  u'district': u'Norwich Free Academy District',
  u'district_code': u'9010022',
  u'education_program': u'',
  u'fax': u'',
  u'grade_1': u'',
  u'grade_10': u'',
  u'grade_11': u'',
  u'grade_12': u'',
  u'grade_2': u'',
  u'grade_3': 

In [22]:
# Record 0 looks right
fixed = fix_record(fixed, "NORWICH", find_district_code("NORWICH")[0])

In [23]:
# What's still missing?
fixed[fixed["school_code"].isnull()]

,drg,drg_distname,school_code,district_name_check
160,H,STAMFORD,None,None
162,I,BRIDGEPORT,None,None


# 4. Stafmord


In [24]:
# 1 looks right:
find_district_code("STAMFORD")[1]

{u'city': u'Stamford',
 u'district': u'Stamford School District',
 u'district_code': u'1350011',
 u'education_program': u'',
 u'fax': u'',
 u'grade_1': u'',
 u'grade_10': u'',
 u'grade_11': u'',
 u'grade_12': u'',
 u'grade_2': u'',
 u'grade_3': u'',
 u'grade_4': u'',
 u'grade_5': u'',
 u'grade_6': u'',
 u'grade_7': u'',
 u'grade_8': u'',
 u'grade_9': u'',
 u'inter_district_magnet': u'',
 u'kindergarten': u'',
 u'phone': u'203-977-4543',
 u'pre_kindergarten': u'',
 u'program_type': u'',
 u'school': u'Stamford School District',
 u'school_code': u'1350011',
 u'state': u'CT',
 u'street': u'888 Washington Boulevard, Box 9310',
 u'type': u'Public School Districts',
 u'website': u'www.stamfordpublicschools.org',
 u'zip': u'06901'}

In [25]:
fixed = fix_record(fixed, "STAMFORD", find_district_code("STAMFORD")[1])

In [26]:
# What's still missing?
fixed[fixed["school_code"].isnull()]

,drg,drg_distname,school_code,district_name_check
162,I,BRIDGEPORT,None,None


# 5. Bridgeport

In [27]:
# 0 looks right
find_district_code("BRIDGEPORT")[0]

{u'city': u'Bridgeport',
 u'district': u'Bridgeport School District',
 u'district_code': u'0150011',
 u'education_program': u'',
 u'fax': u'',
 u'grade_1': u'',
 u'grade_10': u'',
 u'grade_11': u'',
 u'grade_12': u'',
 u'grade_2': u'',
 u'grade_3': u'',
 u'grade_4': u'',
 u'grade_5': u'',
 u'grade_6': u'',
 u'grade_7': u'',
 u'grade_8': u'',
 u'grade_9': u'',
 u'inter_district_magnet': u'',
 u'kindergarten': u'',
 u'phone': u'203-576-7302',
 u'pre_kindergarten': u'',
 u'program_type': u'',
 u'school': u'Bridgeport School District',
 u'school_code': u'0150011',
 u'state': u'CT',
 u'street': u'45 Lyon Terrace, Rm 303',
 u'type': u'Public School Districts',
 u'website': u'www.bridgeportedu.com/',
 u'zip': u'06604'}

In [28]:
fixed = fix_record(fixed, "BRIDGEPORT", find_district_code("BRIDGEPORT")[0])


# Finally generate CSV

In [29]:
# What's still missing?
fixed[fixed["school_code"].isnull()]

,drg,drg_distname,school_code,district_name_check


In [31]:
fixed.to_csv("output/drg.csv", index=False)

In [32]:
# Copy to clean
fixed.to_csv("../clean/drg.csv", index=False)

In [33]:
# What are the DRGS? A- through what?
fixed["drg"].unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'], dtype=object)